In [1]:
import pandas as pd
from __future__ import division
from numbers import Number
import sys, codecs
import numpy as np
import sqlite3
import nltk

import matplotlib.pyplot as plt
%matplotlib inline

# Load data
There are 2 datasets I need:  
* **bgg_ratings_comments_full_r1.db**: SQL db storing all of the reviews for 2000+ games (along with username, rating, game name, and game id) for each review.  
* **bgg_gamelist.csv**: CSV file containing some meta-data for each of the games (one row per game). 

Ultimately I merge these two together and save it in a Dataframe called **df_allgames**.

In [9]:
def import_meta_gamelist(meta_gamelist_filename = 'bgg_gamelist.csv',
                        col_names = ['gameid', 'name', 'n_ratings', 'pic_url']):    
    """ 
    Import the Board Game Meta data from csv file.
    Input: 
        1. meta_gamelist_filename: file name of game list data with meta data (e.g., gameID, etc)
        2. col_names: what you want to name the columns of the dataset
    """

    df_meta_gamelist = pd.read_csv(meta_gamelist_filename) # Read in csv file with meta game info 
    df_meta_gamelist.columns = col_names # prettify column names
    
    return df_meta_gamelist


def import_reviews_from_db(db_filename = "bgg_ratings_comments_full_r1.db",
                          sql_query = "SELECT * FROM data WHERE value!='' AND rating>0"):
    """
    Import data from database file that contains review data (acquired from BGG API).
    Default sql_query extracts all non-empty reviews that contain a rating >0 (this
    eliminates a number of rows that have 0s across all columns)
    """
    connex = sqlite3.connect(df_filename)  # Opens DB file 
    cur = connex.cursor()  # Establish communication with DB 

    sql = sql_query + ";" # SQL query
    df_reviews_and_ratings = pd.read_sql_query(sql, connex) # Read DB data into 
    df_reviews_and_ratings = df_reviews_and_ratings.drop_duplicates() # Drop the duplicate rows

    connex.close() # close connection to db
    
    return df_reviews_and_ratings

def merge_meta_and_review_dfs(df_meta_gamelist, df_reviews_and_ratings):
    """
    Merge meta gamelist df together with reviews df.
    """
    # Merge game meta data with reviews
    df_merged = pd.merge(df_reviews_and_ratings, df_meta_gamelist, 
                               how='left', on='gameid')
    return df_merged

def import_and_merge_datasets(meta_gamelist_filename = 'bgg_gamelist.csv',
                              col_names = ['gameid', 'name', 'n_ratings', 'pic_url'],
                              db_filename = "bgg_ratings_comments_full_r1.db",
                              sql_query = "SELECT * FROM data WHERE value!='' AND rating>0"):
    """
    Implement all previous functions to import meta data and review data and merge
    them together. 
    """
    df_meta_gamelist = import_meta_gamelist(meta_gamelist_filename = 'bgg_gamelist.csv',
                                            col_names = ['gameid', 'name', 
                                                         'n_ratings', 'pic_url'])
    df_reviews_and_ratings = import_reviews_from_db(db_filename = "bgg_ratings_comments_full_r1.db",
                                                    sql_query = "SELECT * FROM data WHERE value!='' AND rating>0")
    df_merged = merge_meta_and_review_dfs(df_meta_gamelist=df_meta_gamelist, 
                                          df_reviews_and_ratings=df_reviews_and_ratings)
    
    return df_merged, df_meta_gamelist
    

In [10]:
# Pull review data from DB and merge with game meta data (df_allgames)
# Also returnt the df_meta_gamelist as bgg_gamelist
df_allgames, bgg_gamelist = import_and_merge_datasets(meta_gamelist_filename = 'bgg_gamelist.csv',
                              col_names = ['gameid', 'name', 'n_ratings', 'pic_url'],
                              db_filename = "bgg_ratings_comments_full_r1.db",
                              sql_query = "SELECT * FROM data WHERE value!='' AND rating>0")

In [11]:
df_allgames.head()

,gameid,username,rating,value,name,n_ratings,pic_url
0,13,sinahero,10.0,The best game in the world,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
1,13,Cayden101,10.0,I've played this game probably close to a hund...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
2,13,Spaceage Polymer,10.0,"I didn't want to give Catan a perfect ten, bec...",Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
3,13,asauve19,10.0,My all time favorite game. Even if you are rol...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...
4,13,Feelie,10.0,One of (if not) the best board game. Ever chan...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...


# Text Cleaning: 
Remove board game names and Tokenize reviews

### Step 1: Make text in reviews all lowercase, and remove almost all board game names from reviews
Sometimes reviewers mention the board game names in reviews (e.g., "I first played *Splendor* with my friends in grad school."). I did not use any entity recognition techniques (like spacy), and therefore name dropping would only introduce noise into my average word vectors. To eliminate noise, I decided to filter (most of) the names out of the reviews. The exceptions included games that had names that can occur in other unrelated words (e.g., 'coup' is in the word 'couple'). 

In [12]:
import nltk

def cleaning1_lowercase_and_remove_bg_names(df_with_reviews = df_allgames,
                                            raw_review_col_name = 'value',
                                            df_gamelist = bgg_gamelist,
                                            game_name_col_df_gamelist = 'name'):
    """
    This function makes all text in reviews lowercase, and removes all
        board game names from review text (except for those on the 
        nameExclude_list - see below). 
    Input: All defaults
        Note: game_name_col_df_gamelist is the column within df_gamelist
        containing the names of the board games. 1 row per game in this df.
    Output: df_with_reviews df with a new column (rev_LC_noNames) 
        containing the reviews 
    """
    
    # Make a list with all the board game names 
    gameName_list = df_gamelist[game_name_col_df_gamelist].unique().tolist() 
    gameName_list = [game.lower() for game in gameName_list] # make lower case
    #print(len(gameName_list)) # debugging

    # Problem names: Names that appear too often in real words. 
    # Don't remove these from reviews. 
    nameExclude_list = ['ys', 'go', 'coup', 'ra', 'goa', 'set', 'pit', 
                        'fuse', 'roma', 'evo', 'aton', 'fits', 'frag', 
                        'pairs', 'rage', 'edo', 'war', 'tak']
    #[token for token in tokens if token not in en_stop]
    gameName_list = [game for game in gameName_list if game not in nameExclude_list]
    #print(len(gameName_list)) # debugging
    
    ##### Remove all board game names from the reviews. 

    ### Make reviews all lower case and replace game names with nothing
    # make raw reviews lowercase and save to new column named 'comments'
    # Note: this doesn't take long to run. 
    df_with_reviews['rev_LC_noNames'] = df_with_reviews[raw_review_col_name].apply(lambda x: x.lower()) 
    
    # Replace game names with ""
    # Note: This loop takes a while (because there are 2000+ games to loop through)
    for game in gameName_list:
        df_with_reviews['rev_LC_noNames'] = df_with_reviews['rev_LC_noNames'].apply(lambda x: x.replace(game, "")) 
    
    return df_with_reviews


def cleaning2_tokenize_words(df_with_reviews = df_allgames):
    '''
    Parse reviews into words. (Need to import nltk first.)
    Output: Adds a column (rev_cleaned) to df_with_reviews containing 
    tokenized reviews.
    '''

    ### Tokenize words in reviews: 
    df_with_reviews['rev_cleaned'] = df_with_reviews['rev_LC_noNames'].apply(lambda x: nltk.word_tokenize(x))
    
    return df_with_reviews


def cleaning3_remove_stopwords(df_with_reviews = df_allgames,
                              nonfeatures = ['star', 'nostar']):
    '''
    Cleaning Step 3. Remove stop words and words not likely to be features.
    Output: Modified rev_cleaned column within df_with_reviews.
    '''
    en_stop = set(nltk.corpus.stopwords.words('english')) # set stop words
    
    # updates set of stop words with the ones I wanted to add (i.e., 
    # very frequently used emoticons)
    en_stop.update(nonfeatures) 
    
    # Remove stop words:
    df_with_reviews['rev_cleaned'] = df_with_reviews['rev_cleaned'].apply(lambda review: [word for word in review if word not in en_stop])
    
    return df_with_reviews


In [13]:
### Function applies all cleaning steps to df.
def clean_reviews(df_with_reviews = df_allgames,
                  raw_review_col_name = 'value',
                  df_gamelist = bgg_gamelist,
                  game_name_col_df_gamelist = 'name',
                  nonfeatures = ['star', 'nostar']):
    """
    Applies all 3 cleaning steps to clean raw reviews in df_with_reviews. 
    Creates 2 new columns within df_with_reviews: 
        1. 'rev_LC_noNames': untokenized, lowercase reviews with board game
            names removed. This should be used later for Sentiment 
            Analysis and keyword search (if you're into that). 
        2. 'rev_cleaned': cleaned reviews (ready for Word2Vec).
    """
    import nltk
    
    # Run Step 1: Lowercase reviews and remove board game names.
    df_with_reviews = cleaning1_lowercase_and_remove_bg_names(df_with_reviews = df_with_reviews,
                                            raw_review_col_name = raw_review_col_name,
                                            df_gamelist = df_gamelist,
                                            game_name_col_df_gamelist = game_name_col_df_gamelist)
    # Run Step 2: Tokenize reviews into words.
    df_with_reviews = cleaning2_tokenize_words(df_with_reviews = df_with_reviews)

    # Apply Cleaning Step 3. Remove stop words. 
    df_with_reviews = cleaning3_remove_stopwords(df_with_reviews = df_with_reviews,
                                             nonfeatures = nonfeatures)
    return df_with_reviews


In [15]:
import nltk

# Clean text in reviews
df_allgames = clean_reviews(df_allgames)

df_allgames.head()

,gameid,username,rating,value,name,n_ratings,pic_url,rev_LC_noNames,rev_cleaned
0,13,sinahero,10.0,The best game in the world,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,the best game in the world,"[best, game, world]"
1,13,Cayden101,10.0,I've played this game probably close to a hund...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,i've played this game probably close to a hund...,"['ve, played, game, probably, close, hundred, ..."
2,13,Spaceage Polymer,10.0,"I didn't want to give Catan a perfect ten, bec...",Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,"i didn't want to give a perfect ten, because ...","[n't, want, give, perfect, ten, ,, 's, even, f..."
3,13,asauve19,10.0,My all time favorite game. Even if you are rol...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,my all time favorite game. even if you are rol...,"[time, favorite, game, ., even, rolling, bad, ..."
4,13,Feelie,10.0,One of (if not) the best board game. Ever chan...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,one of (if not) the best board game. ever chan...,"[one, (, ), best, board, game, ., ever, changi..."


In [16]:
#### Save Output as pickle.  

import pickle

filename = 'bgg_ratings_reviews_full_posttextcleaning.pkl'
outfile = open(filename, 'wb')
pickle.dump(df_allgames, outfile)
outfile.close()


In [3]:
import pickle

## Pickle Load: 
filename = 'bgg_ratings_reviews_full_posttextcleaning.pkl'

infile = open(filename, 'rb')
df_allgames = pickle.load(infile)
infile.close()

df_allgames.head()

,gameid,username,rating,value,name,n_ratings,pic_url,rev_LC_noNames,rev_cleaned
0,13,sinahero,10.0,The best game in the world,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,the best game in the world,"[best, game, world]"
1,13,Cayden101,10.0,I've played this game probably close to a hund...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,i've played this game probably close to a hund...,"['ve, played, game, probably, close, hundred, ..."
2,13,Spaceage Polymer,10.0,"I didn't want to give Catan a perfect ten, bec...",Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,"i didn't want to give a perfect ten, because ...","[n't, want, give, perfect, ten, ,, 's, even, f..."
3,13,asauve19,10.0,My all time favorite game. Even if you are rol...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,my all time favorite game. even if you are rol...,"[time, favorite, game, ., even, rolling, bad, ..."
4,13,Feelie,10.0,One of (if not) the best board game. Ever chan...,Catan,77596,https://cf.geekdo-images.com/micro/img/e0y6Bog...,one of (if not) the best board game. ever chan...,"[one, (, ), best, board, game, ., ever, changi..."


In [7]:
# Save dataframe to database. 

# connex = sqlite3.connect('bgg_ratings_reviews_full_posttextcleaning.db')  # Opens DB file (if file doesn't exist, it creates it)
# cur = connex.cursor()  # Establish communication with DB 

# # if file exists, it overwrites it:
# df_allgames.to_sql(name="data", con=connex, if_exists="replace", index=False) 